<a href="https://colab.research.google.com/github/ramakrishna0008/Model_practice/blob/master/LangChain_OpenAI_Pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!git clone https://github.com/ramakrishna0008/Model_practice.git

fatal: destination path 'Model_practice' already exists and is not an empty directory.


In [21]:
!pip -q install langchain openai tiktoken PyPDF2 faiss-cpu

In [24]:
import os

from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass('Enter the OPENAI_API_KEY: ')

Enter the OPENAI_API_KEY: ··········


In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.297
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [ ]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [ ]:
# location of the pdf file/files.
doc_reader = PdfReader('/content/Model_practice/Taj Mahal.pdf')

In [ ]:
doc_reader

In [ ]:
# read data from the file and put them into a variable called raw_text
raw_text = ''
for i, page in enumerate(doc_reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [ ]:
len(raw_text)

1531

In [ ]:
raw_text[:100]

'The\nTaj\nMahal\nis\none\nof\nthe\nmost\niconic\nand\nrecognizable\nlandmarks\nin\nthe\nworld.\nIt\nis\nlocated\nin\nAg'

In [ ]:
# Splitting up the text into smaller chunks for indexing
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 200,
    chunk_overlap  = 50, #striding over the text
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
len(texts)

10

In [ ]:
texts[5]

'completed\nlater.\n4.\nSymbol\nof\nLove:\n-\nOften\nreferred\nto\nas\nthe\n"Symbol\nof\nLove,"\nthe\nTaj\nMahal\nis\na\ntestament\nto\nthe\ndeep\nlove\nand\ngrief\nof\nShah\nJahan\nfor\nhis\nwife\nMumtaz.\nIt\nis\nsaid\nthat\nthe'

In [ ]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [ ]:
docsearch = FAISS.from_texts(texts, embeddings)

RateLimitError: ignored

In [ ]:
docsearch.embedding_function

In [ ]:
query = "The Taj Mahal was built by?"
docs = docsearch.similarity_search(query)

In [ ]:
len(docs)

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [ ]:
chain = load_qa_chain(OpenAI(),
                      chain_type="stuff") # we are going to stuff all the docs in at once

In [ ]:
# check the prompt
chain.llm_chain.prompt.template

In [ ]:
query = "The location of Taj Mahal?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

### Reference:
https://www.youtube.com/watch?v=ZzgUqFtxgXI&ab_channel=SamWitteveen